In [19]:
import numpy as np
import mne 
import csv
import os

In [20]:
# IN
dir = '/home/SharedFiles/Data/HospitalData/EEG/skcc'
edf_dir = os.path.join(dir, 'edf')
annot_dir = os.path.join(dir, 'annotation_renew')

# OUT
curr_dir = os.path.abspath(os.getcwd())
data_dir = os.path.join(curr_dir, 'data')
ictal_dir = os.path.join(data_dir, 'ictal')
interictal_dir = os.path.join(data_dir, 'interictal') 
    
for dir_ in (ictal_dir, interictal_dir):
    if not os.path.exists(dir_):
        os.makedirs(dir_)

In [21]:
# all_channels = [
#     'Fp1-AVG',
#     'F3-AVG',
#     'C3-AVG',
#     'P3-AVG',
#     'Fp2-AVG',
#     'F4-AVG',
#     'C4-AVG',
#     'P4-AVG',
#     'F7-AVG',
#     'T1-AVG',
#     'T3-AVG',
#     'T5-AVG',
#     'O1-AVG',
#     'F8-AVG',
#     'T2-AVG',
#     'T4-AVG',
#     'T6-AVG',
#     'O2-AVG',
#     'Fz-AVG',
#     'Cz-AVG',
#     'Pz-AVG',
#     ] 
channels = ['Fp1-AVG', 'Fp2-AVG', 'T5-AVG', 'T6-AVG']

patient_names= ['SK02_1','SK02_2','SK03','SK05','SK06','SK07','SK08','SK09','SK10','SK11','SK14_1','SK14_2','SK17','SK21']
# train_fold_list = [[1,2,3,6,7],[1,3],[1,2,5,6,7],[2,3],[1,3,4,5]]
event_level_testsets = [[[6,8]],[[]],[[15,17]],[[4,6]],[[7,9],[20,22]],[[5,7],[10,12]],[[0,2]],[[7,9]],[[18,20]],
                              [[14,16],[19,20.5]],[[9,11]],[[]],[[12,14],[14,16]],[[8,10]] ] # test folds

In [22]:
def annotater(annot_path):
    with open(annot_path, mode='r') as file:
        csvFile = csv.reader(file)
        marks = [int(float(line[0])) for line in csvFile]
        marks.sort()
    starts = marks[::2]
    ends = marks[1::2]

    return starts, ends


In [23]:
def detection_segmenter(raw, ictal_starts, ictal_ends):
    if len(ictal_starts) != len(ictal_ends):
        print('Error: ictal starts and ends lengths are not equivalent')
        return None
    
    tot_time = np.round(raw.times[-1])
    print('tot_time', tot_time)
    ictals, interictals = [], []
    for i in range(len(ictal_starts)):
        # if first ictal
        if i == 0:
            interictal_end = ictal_starts[0] - buffer_len
            if interictal_end < 0+seq_len:
                continue
            interictal_start = 0
        
        # if last ictal
        elif i == len(ictal_starts)-1:
            interictal_start = ictal_ends[-1] + buffer_len
            if interictal_start > tot_time:
                break
            interictal_end = tot_time

        else:
            interictal_start = ictal_ends[i]
            interictal_end = ictal_starts[i+1]
        
        # crop and append
        ictals.append(raw.copy().crop(
            tmin=ictal_starts[i], tmax=ictal_ends[i], include_tmax=False))
        interictals.append(raw.copy().crop(
            tmin=interictal_start, tmax=interictal_end, include_tmax=False))
                           
        print('interictal cuts:', interictal_start, interictal_end)
        print('ictal cuts:', ictal_starts[i], ictal_ends[i])

    return ictals, interictals


In [28]:
buffer_len = 10 * 60 #sec
resample_sfreq = 200.0
notch_freqs = 30.0
seq_len = 4 #sec

for file in os.listdir(edf_dir):
    name = file.split('.')
    if name[1].upper() == 'EDF' and name[0] in patient_names:
        pname = name[0]
        print('---', pname)
        
        raw = mne.io.read_raw_edf(os.path.join(edf_dir, file))
        
        # channels
        raw = raw.pick(channels)
        # print(raw.info)

        # annotate
        annot_path = os.path.join(annot_dir, pname + '.txt')
        starts, ends = annotater(annot_path)

        # segment
        ictals, interictals = detection_segmenter(raw, starts, ends)
        
        d_ict = {'ictal': ictals, 'interictal': interictals}
        for type_ in d_ict: 
            crops = d_ict[type_]
            for i, crop in enumerate(crops):
                # resample
                # crop = crop.resample(resample_sfreq, n_jobs=-1)

                # arrayify
                arr = crop.get_data()
                # print(arr.shape)

                # save
                fname = pname + '_' + type_ + '_' + str(i).zfill(len(str(len(crops)))) + '.npy'
                fpath = os.path.join(data_dir, type_, fname)
                # np.save(fpath, arr, allow_pickle=False)
                print(fpath)



--- SK02_1
Extracting EDF parameters from /home/SharedFiles/Data/HospitalData/EEG/skcc/edf/SK02_1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
tot_time 84350.0
ictal cuts: 3466 3508
interictal cuts: 0 2866
ictal cuts: 25872 25899
interictal cuts: 26499 84350.0
/home/SharedFiles/Projects/EEG/skbp_Youn/data/ictal/SK02_1_ictal_0.npy
/home/SharedFiles/Projects/EEG/skbp_Youn/data/ictal/SK02_1_ictal_1.npy
/home/SharedFiles/Projects/EEG/skbp_Youn/data/interictal/SK02_1_interictal_0.npy
/home/SharedFiles/Projects/EEG/skbp_Youn/data/interictal/SK02_1_interictal_1.npy
--- SK02_2
Extracting EDF parameters from /home/SharedFiles/Data/HospitalData/EEG/skcc/edf/SK02_2.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
tot_time 80927.0
ictal cuts: 5772 5833
interictal cuts: 0 5172
ictal cuts: 79740 79800
interictal cuts: 80400 80927.0
/home/SharedFiles/Projects/EEG/skbp_Youn/data/ictal/SK02_2_ictal_0.npy
/home/SharedF

In [ ]:
# TODO: Event level: take out event level portions (event level test sets * 60*60 #s) after annotating/segmenting ictals

: 